# 공격성 규범 점수

## 평가내역에 따른 공격성 규범 산출

In [1]:
import sys
print(sys.executable)   # 예: C:\ProgramData\anaconda3\python.exe
print(sys.path)         # 패키지 탐색 경로(여기에 site-packages가 있어야 import 가능)

/opt/anaconda3/bin/python
['/Users/mac/insight_/aws_etl/schoolfriends', '/opt/anaconda3/lib/python312.zip', '/opt/anaconda3/lib/python3.12', '/opt/anaconda3/lib/python3.12/lib-dynload', '', '/opt/anaconda3/lib/python3.12/site-packages', '/opt/anaconda3/lib/python3.12/site-packages/aeosa', '/opt/anaconda3/lib/python3.12/site-packages/setuptools/_vendor']


In [3]:
import awswrangler as wr

In [32]:
df = wr.athena.read_sql_query(
    """
    select a.school_code, a.school_name, b.school_grade, b.school_num, b.class_name, c.psy_name, c.psy_code, c.close_yn, f.student_num, d.target_code, e.user_testing_no, e.question_no, e.student_code 
    from school_info a
    inner join school_class b on a.school_code = b.school_code
    inner join psy_class c on b.class_code = c.class_code
    inner join psy_target d on c.class_code = d.class_code and c.psy_code = d.psy_code
    inner join at_user_testing_paper_pn e on d.user_testing_no = e.user_testing_no
    inner join school_student f on d.target_code = f.student_code and d.class_code = f.class_code
    where c.close_yn = 'Y' and e.question_no in ('4','5','6','7','14')
    """
    , database="schoolfriends"
)

In [33]:
df

,school_code,school_name,school_grade,school_num,class_name,psy_name,psy_code,close_yn,student_num,target_code,user_testing_no,question_no,student_code
0,AC0001,학지초등학교,"""6""","""4""",우리 또래 반,"""25년 7월 1회차""",P20250716BE86,Y,"""20""",AC000120256042D2E020,"""557b6318ba464b62be79""",4,AC000120256042D2E003
1,AC0001,학지초등학교,"""6""","""4""",우리 또래 반,"""25년 7월 1회차""",P20250716BE86,Y,"""20""",AC000120256042D2E020,"""557b6318ba464b62be79""",5,AC000120256042D2E003
2,AC0001,학지초등학교,"""6""","""4""",우리 또래 반,"""25년 7월 1회차""",P20250716BE86,Y,"""20""",AC000120256042D2E020,"""557b6318ba464b62be79""",6,AC000120256042D2E003
3,AC0001,학지초등학교,"""6""","""4""",우리 또래 반,"""25년 7월 1회차""",P20250716BE86,Y,"""20""",AC000120256042D2E020,"""557b6318ba464b62be79""",6,AC000120256042D2E009
4,AC0001,학지초등학교,"""6""","""4""",우리 또래 반,"""25년 7월 1회차""",P20250716BE86,Y,"""20""",AC000120256042D2E020,"""557b6318ba464b62be79""",6,AC000120256042D2E012
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5067,TC0461,목포부주초등학교,"""6""","""7""","""6학년7반""","""25년 7월 1회차""",P20250722BAB6,Y,"""1""",TC046120256076567001,eeaf384f0e31454cb6d0,6,""""""
5068,TC0461,목포부주초등학교,"""6""","""7""","""6학년7반""","""25년 7월 1회차""",P20250722BAB6,Y,"""1""",TC046120256076567001,eeaf384f0e31454cb6d0,7,""""""
5069,TC0461,목포부주초등학교,"""6""","""7""","""6학년7반""","""25년 7월 1회차""",P20250722BAB6,Y,"""1""",TC046120256076567001,eeaf384f0e31454cb6d0,14,TC046120256076567012
5070,TC0461,목포부주초등학교,"""6""","""7""","""6학년7반""","""25년 7월 1회차""",P20250722BAB6,Y,"""1""",TC046120256076567001,eeaf384f0e31454cb6d0,14,TC046120256076567021


In [34]:
df.columns

Index(['school_code', 'school_name', 'school_grade', 'school_num',
       'class_name', 'psy_name', 'psy_code', 'close_yn', 'student_num',
       'target_code', 'user_testing_no', 'question_no', 'student_code'],
      dtype='object')

In [40]:
# 지명수 구하는 함수
def count_point(df, student_list, item_no) :
    import pandas as pd
    if df['psy_code'].nunique() > 1 :
        raise ValueError('psy_code 컬럼에 여러 값이 존재합니다.')
    else :
        temp_df = df[df['question_no'].isin(item_no)] # 해당 문항 필터링
        count_list = []
        for student in student_list :
            if len(temp_df[temp_df['student_code']==student]) >= 1 :
                freq = len(temp_df[temp_df['student_code']==student])
            elif len(temp_df[temp_df['student_code']==student]) == 0 :
                freq = 0
            count_list.append(freq)
        count_point_df = pd.DataFrame({'student_code': student_list, 'num_point': count_list})
        return count_point_df, df['psy_code'].iloc[0]

# 학생 리스트 구하는 함수
def student_list(df) :
    student_list = df['target_code'].unique().tolist()
    return student_list, df['psy_code'].iloc[0]

In [41]:
df = df[df['psy_name'].str.contains('이아린비활')]
print(df)

    school_code school_name school_grade school_num class_name  \
745      AC0001      학지초등학교          "5"        "1"    "5학년1반"   
746      AC0001      학지초등학교          "5"        "1"    "5학년1반"   
747      AC0001      학지초등학교          "5"        "1"    "5학년1반"   
748      AC0001      학지초등학교          "5"        "1"    "5학년1반"   
749      AC0001      학지초등학교          "5"        "1"    "5학년1반"   
..          ...         ...          ...        ...        ...   
935      AC0001      학지초등학교          "5"        "1"    "5학년1반"   
936      AC0001      학지초등학교          "5"        "1"    "5학년1반"   
937      AC0001      학지초등학교          "5"        "1"    "5학년1반"   
938      AC0001      학지초등학교          "5"        "1"    "5학년1반"   
939      AC0001      학지초등학교          "5"        "1"    "5학년1반"   

               psy_name       psy_code close_yn student_num  \
745  "25년 7월 2회차_이아린비활"  P202507183B0A        Y        "22"   
746  "25년 7월 2회차_이아린비활"  P202507183B0A        Y        "22"   
747  "25년 7월 2회차_이

In [42]:
student_list_obj, psy_code = student_list(df)
print(student_list_obj, psy_code)
print(len(student_list_obj))

['AC000120255019FA1022', 'AC000120255019FA1021', 'AC000120255019FA1020', 'AC000120255019FA1019', 'AC000120255019FA1018', 'AC000120255019FA1017', 'AC000120255019FA1016', 'AC000120255019FA1014', 'AC000120255019FA1013', 'AC000120255019FA1012', 'AC000120255019FA1011', 'AC000120255019FA1010', 'AC000120255019FA1009', 'AC000120255019FA1008', 'AC000120255019FA1007', 'AC000120255019FA1006', 'AC000120255019FA1005', 'AC000120255019FA1004', 'AC000120255019FA1003', 'AC000120255019FA1002', 'AC000120255019FA1001'] P202507183B0A
21


In [43]:
aggression_point_df, psy_code = count_point(df, student_list_obj, ['4','5','6','7'])
n14_point_df, psy_code = count_point(df, student_list_obj, ['14'])
print(aggression_point_df, psy_code)
print(n14_point_df, psy_code)

            student_code  num_point
0   AC000120255019FA1022          4
1   AC000120255019FA1021          8
2   AC000120255019FA1020          4
3   AC000120255019FA1019          8
4   AC000120255019FA1018          9
5   AC000120255019FA1017          9
6   AC000120255019FA1016          2
7   AC000120255019FA1014          8
8   AC000120255019FA1013          4
9   AC000120255019FA1012          9
10  AC000120255019FA1011          7
11  AC000120255019FA1010          3
12  AC000120255019FA1009          3
13  AC000120255019FA1008          7
14  AC000120255019FA1007          5
15  AC000120255019FA1006         10
16  AC000120255019FA1005          9
17  AC000120255019FA1004          1
18  AC000120255019FA1003          7
19  AC000120255019FA1002          3
20  AC000120255019FA1001         11 P202507183B0A
            student_code  num_point
0   AC000120255019FA1022          1
1   AC000120255019FA1021          1
2   AC000120255019FA1020          1
3   AC000120255019FA1019          0
4   AC00012025

In [44]:
def aggression_norms(df) :
    import pandas as pd
    aggression_point_df, psy_code = count_point(df, student_list_obj, ['4','5','6','7'])
    n14_point_df, psy_code = count_point(df, student_list_obj, ['14'])

    aggression_norms_df = pd.merge(aggression_point_df, n14_point_df, on='student_code', suffixes=('_agg', '_14no'))
    aggression_norms_df['agg*14no_num_point'] = aggression_norms_df['num_point_agg'] * aggression_norms_df['num_point_14no']
    aggression_norms_df.sort_values(by='agg*14no_num_point', ascending=False, inplace=True)
    
    top5_aggression_norms_df = aggression_norms_df.head(5)
    top5_mean = top5_aggression_norms_df['agg*14no_num_point'].mean()
    total_mean = aggression_norms_df['agg*14no_num_point'].mean()
    weighted_mean = 0.4 * (top5_mean) + 0.6 * (total_mean)
    weighted_mean = 4 * ((min(max(weighted_mean, 0), 67)-0)/67)+1
    result_df = pd.DataFrame({'psy_code': [psy_code], 'top5_mean': [top5_mean], 'total_mean': [total_mean], 'weighted_mean': [weighted_mean]})
    return result_df

In [45]:
result_df = aggression_norms(df)
print(result_df)

        psy_code  top5_mean  total_mean  weighted_mean
0  P202507183B0A       27.8   11.571429        2.07838


## raw 쿼리 결과 -> 그룹별 채점 결과 변환

# 쿼리 결과를 딕셔너리로 만들어
# 각 객체(키:df) 식으로 변환 처리